In [3]:
import numpy as np
import pandas as pd
from scipy.stats import norm

## Black Scholes Merton Equation

$$ \frac{\partial{V}}{\partial{t}} + (r-D)S\frac{\partial{V}}{\partial{S}} + \frac{1}{2}\sigma^{2}S^{2}\frac{\partial^2{V}}{\partial{S^2}} - rV = 0 $$

The above Black Scholes Merton equation can be transformed to the following equation.

In [4]:
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

# Finite Difference Method

What is the finite difference method?\
&nbsp;&nbsp;&nbsp; It is a numerical technique for solving differential equations by approximating them at each mesh point.

For the  Black Scholes Merton equation, the x-axis represents each time step and the y-axis represents the value of the underlyig asset where T (last t) is the time at expiry and S* (maximum asset price) is large even though practically there is no limit for asset price. As can be seen from the figure below.

<p align="center"><img title="mesh" alt="Mesh" src="http://www.goddardconsulting.ca/image-files/finiteelement_FEMGrid.jpg" width="500/">

## Let's start with the explicit scheme for European options.
It starts at the end (t=T) and then steps backward through time to reach present using three price points at the step before.

#################### add a pic ###############################


We start by Taylor-series approximations:
$$ V(S + \delta S,t + \delta t) = V(S,t) + \frac{\partial{V}}{\partial{S}}\delta S + \frac{\partial{V}}{\partial{t}}\delta t + \frac{1}{2}\frac{\partial^2{V}}{\partial{S^2}}\delta S^{2} + \frac{1}{2}\frac{\partial^2{V}}{\partial{t^2}}\delta t^{2} + \frac{\partial^2{V}}{\partial{S}\partial{t}}\delta S\delta t + O(\delta S^3,\delta t^3)$$

Let's consider
$$ V(S,t + \delta t) = V(S,t) + \frac{\partial{V}}{\partial{t}}\delta t + O(\delta t^2)$$

by rearranging, we get

$$ \Theta = \frac{\partial{V}}{\partial{t}} = \frac{V(S,t + \delta t) - V(S,t)}{\delta t} + O(\delta t^2) \approx \frac{V(S,t + \delta t) - V(S,t)}{\delta t} $$ 

or in finite difference form

$$ \frac{\partial{V}}{\partial{t}}(n\delta S, m \delta t) \sim \frac{V^m_n - V^{m-1}_n}{\delta t} $$ 

Next we consider

$$ V(S + \delta S,t) = V(S,t) + \frac{\partial{V}}{\partial{S}}\delta S + \frac{1}{2}\frac{\partial^2{V}}{\partial{S^2}}\delta S^{2} + O(\delta S^3) \tag{1}$$
$$ V(S - \delta S,t) = V(S,t) - \frac{\partial{V}}{\partial{S}}\delta S + \frac{1}{2}\frac{\partial^2{V}}{\partial{S^2}}\delta S^{2} - O(\delta S^3) \tag{2}$$

$(1)-(2):$

$$ V(S + \delta S,t) - V(S - \delta S,t) = 2\frac{\partial{V}}{\partial{S}}\delta S + O(\delta S^3) $$

rearrange the above to get

$$ \Delta = \frac{\partial{V}}{\partial{S}} = \frac{V(S + \delta S,t) - V(S - \delta S,t)}{2 \delta S} + O(\delta S^3) $$ 

$$\frac{\partial{V}}{\partial{S}}(n\delta S, m \delta t) \sim \frac{V^m_{n+1} - V^m_{n-1}}{2 \delta S} $$

$(1)+(2)$

$$ V(S + \delta S,t) + V(S - \delta S,t) = 2V(S,t) + \frac{\partial^2{V}}{\partial{S^2}}\delta S^{2} + O(\delta S^3) $$

rearrange the above to get

$$ \Gamma = \frac{\partial^2{V}}{\partial{S^2}} = \frac{V(S + \delta S,t) - 2V(S,t) + V(S - \delta S,t)}{\delta S^2} + O(\delta S^4)$$
$$ \frac{\partial^2{V}}{\partial{S^2}}(n\delta S, m \delta t) \sim \frac{V^m_{n-1} - 2V^m_n + V^m_{n+1}}{\delta S^2}$$

Finally, we combine the tree previous results and get
$$ V^{m-1}_n = \alpha_n V^m_{n+1} + \beta_n V^m_{n} + \gamma_n V^m_{n-1} \tag{*}$$

$$
\begin{aligned}
\alpha_n &= \frac{1}{2}(n^2\sigma^2-n(r-D))\delta t \\

\beta_n &= 1-(r+n^2\sigma^2)\delta t \\

\gamma_n &= \frac{1}{2}(n^2\sigma^2+n(r-D))\delta t
\end{aligned}
$$

For boundary conditions:\
For t = T, the values of options are their payoff at different asset price steps.

$$V^M_n = payoff(n\delta S,E)$$

For S = 0, the values of options are the present value of the next future step (keep in mind that we are moving backward).
$$V^{m-1}_0 = (1-r\delta t)V^m_0$$

For S = S* (large S), we have to use gamma ($\Gamma$). We know that at S*, $\Gamma$ = 0 because the rate of change of the rate of change in option values must be zero meaning that the rate of change of the rate of change doesn't effect by the price of the underlying asset anymore, so we get
$$\Gamma = \frac{V^m_{n-1} - 2V^m_N + V^m_{n+1}}{\delta S^2} = 0 $$
$$ V^m_{N+1} = 2V^m_N - V^m_{N-1}$$
subsitute this in the $(*)$ to get
$$ V^{m-1}_N =  (\alpha_N - \gamma_N) V^m_{N-1} + (\beta_N + 2\gamma_N) V^m_{N} $$

In [5]:
#All inputs
S_star = 200      #Stock price at the infinity
T = 1             #Expiry date
E = 100           #Strike price or Exercise price
D = 0.00          #Dividend
r = 0.05          #risk free rate
sigma = 0.2       #volatility
ds = S_star/100   #Price increment
a = 0.9/sigma**2/100**2   #for stability
dt = T/(int(T/a)+1) #Time increment

#Payoff function
def payoff(price,exercise,option):
    if option == "call":
        return max(price - exercise,0) #Call option: return max(price - exercise,0) ||| Put option: max(exercise - price,0)
    if option == "put":
        return max(exercise - price,0)

#Check Conditional Stability for Explicit Scheme
# if dt <= 1/(sigma*N)**2:
#     print("Pass")
# else:
#     print("Recheck Conditional Stability")

In [6]:
# European Options, Explicit Scheme
def FDM_BS_European_Ex(T,E,ds,dt,D,r,option):
    N = int(S_star/ds)
    M = int(T/dt) + 1
    
    V = np.zeros((N+1,M+1)) # build a zero matrix

    #Final Payoff Condition (t == T)
    for n in range(N+1):
        V[n][M] = payoff(n*ds,E,option)

    #Differential Equation
    a_N = 0.5*((N**2)*(sigma**2)-N*(r-D))*dt
    b_N = 1-(r+(N**2)*(sigma**2))*dt
    c_N = 0.5*((N**2)*(sigma**2)+N*(r-D))*dt

    for m in range(M,0,-1):
        for n in range(1,N,1):
            a_n = 0.5*((n**2)*(sigma**2)-n*(r-D))*dt
            b_n = 1-(r+(n**2)*(sigma**2))*dt
            c_n = 0.5*((n**2)*(sigma**2)+n*(r-D))*dt
            
            V[n,m-1] = a_n*V[n-1,m] + b_n*V[n,m] + c_n*V[n+1,m]

        #Boundary Condition at S == S_star
        V[N,m-1] = (a_N - c_N)*V[N-1,m] + (b_N + 2*c_N)*V[N,m]     #Call option & Put Option(For put options this is close to 0)

        #Boundary Condition  at S == 0
        b_0 = 1-(r*dt)
        V[0][m-1] = V[0][m]*b_0

    #np.set_printoptions(precision=6)
    return V

## The explicit scheme for American options
For American options, we can exercise an option before the expiry, so the value of the option must not be less than its payoff because of the no-arbitrage argument.

$$ V \ge  payoff(S,E) $$

If the value of the European option is less than the payoff at expiry $ V_{euro} \le payoff(S,E) $ then the value of the American option that has the same property is the payoff $ V_{amer} = payoff(S,E) $. We apply this to the FDM for European options to get American options.

In [7]:
# American Options, Explicit Scheme
def FDM_BS_American_Ex(T,E,ds,dt,D,r,option):
    N = int(S_star/ds)
    M = int(T/dt) + 1
    
    V = np.zeros((N+1,M+1)) # build a zero matrix
    #Final Payoff Condition (t == T)
    for n in range(N+1):
        V[n][M] = payoff(n*ds,E,option)

    #Differential Equation
    a_N = 0.5*((N**2)*(sigma**2)-N*(r-D))*dt
    b_N = 1-(r+(N**2)*(sigma**2))*dt
    c_N = 0.5*((N**2)*(sigma**2)+N*(r-D))*dt

    for m in range(M,0,-1):
        for n in range(0,N,1):
            a_n = 0.5*((n**2)*(sigma**2)-n*(r-D))*dt
            b_n = 1-(r+(n**2)*(sigma**2))*dt
            c_n = 0.5*((n**2)*(sigma**2)+n*(r-D))*dt
            
            V[n,m-1] = a_n*V[n-1,m] + b_n*V[n,m] + c_n*V[n+1,m]
            

        #Boundary Condition at S == S_star
        V[N,m-1] = (a_N - c_N)*V[N-1,m] + (b_N + 2*c_N)*V[N,m]     #Call option & Put Option(For put options this is close to 0)

        #Boundary Condition  at S == 0
        b_0 = 1-(r*dt)
        V[0][m-1] = V[0][m]*b_0

    for m in range(M,-1,-1):
        for n in range(0,N+1,1):
            b_0 = 1-(r*dt)
            V[n,m] = np.maximum( payoff(n*ds,E,option)*b_0 , V[n,m] )

    return V

## The implicit scheme for European options

For the implicit scheme, we move forward through each time step. From the Taylor-series approximations, we get

<!-- %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% add pic %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% -->

$$ \frac{\partial{V}}{\partial{t}}(n\delta S, m \delta t) \sim \frac{V^m_n - V^{m-1}_n}{\delta t} $$ 

$$\frac{\partial{V}}{\partial{S}}(n\delta S, m \delta t) \sim \frac{V^{m-1}_{n+1} - V^{m-1}_{n-1}}{2 \delta S} $$

$$ \frac{\partial^2{V}}{\partial{S^2}}(n\delta S, m \delta t) \sim \frac{V^{m-1}_{n-1} - 2V^{m-1}_n + V^{m-1}_{n+1}}{\delta S^2}$$

combine the above results, we have
$$ V^{m}_n = a_n V^{m-1}_{n-1} + b_n V^{m-1}_{n} + c_n V^{m-1}_{n+1} \tag{*}$$

$$
\begin{aligned}
a_n &= -\frac{1}{2}(n^2\sigma^2-n(r-D))\delta t \\

b_n &= 1+(r+n^2\sigma^2)\delta t \\

c_n &= -\frac{1}{2}(n^2\sigma^2+n(r-D))\delta t
\end{aligned}
$$

For boundary conditions:\
For t = T, the values of options are their payoff at different asset price steps.

$$V^M_n = payoff(n\delta S,E)$$

For S = 0, the values of options are the present value of the next future step (keep in mind that we are moving forward).
$$V^{m}_0 = (1+r\delta t)V^{m-1}_0$$

For S = S* (large S), we still have to use gamma ($\Gamma$). We obtain

$$\Gamma = \frac{V^{m-1}_{n-1} - 2V^{m-1}_N + V^{m-1}_{n+1}}{\delta S^2} = 0 $$
$$ V^{m-1}_{N+1} = 2V^{m-1}_N - V^{m-1}_{N-1}$$

subsitute this in the $(*)$ to get
$$ V^{m-1}_N =  a_N V^{m-1}_{N-1} + b_N V^{m-1}_{N} $$

Finally, we get a linear system to solve
$$
\begin{aligned}
V^m_1 &= a_1 V^{m-1}_0 + b_1 V^{m-1}_1 + c_1 V^{m-1}_2 \\
V^m_2 &= a_2 V^{m-1}_1 + b_2 V^{m-1}_2 + c_2 V^{m-1}_3 \\
\vdots \\
V^m_{N-1} &= a_{N-1} V^{m-1}_{N-2} + b_{N-1} V^{m-1}_{N-1} + c_{N-1} V^{m-1}_N \\
\end{aligned}
$$

or

$$
\begin{bmatrix}
    b_0     & c_0   & \ldots    & \ldots    & \ldots  & 0      \\
    a_1     & b_1   & c_1       & 0         & \ddots  & \vdots \\
    0       & a_2   & b_2       & c_2       & 0       & \vdots \\
    \vdots  & \ddots& \ddots    & \ddots    & \ddots  & 0      \\
    \vdots  & \ddots& 0         & a_{N-1}   & b_{N-1} & c_{N-1}\\
    0       & \ldots& \ldots    & 0         & a_N     & b_N    \\
\end{bmatrix}
\times
\begin{bmatrix}
V^{m-1}_0 \\ 
V^{m-1}_1 \\
V^{m-1}_2 \\
V^{m-1}_3 \\
\vdots    \\
V^{m-1}_{N-1}\\
V^{m-1}_{N}\\
\end{bmatrix}
=
\begin{bmatrix}
V^{m}_0 \\ 
V^{m}_1 \\
V^{m}_2 \\
V^{m}_3 \\
\vdots    \\
V^{m}_{N-1}\\
V^{m}_{N}\\
\end{bmatrix}
$$

solve the linear system to get the result.

In [9]:
# European Options, Implicit Scheme
def FDM_BS_European_Im(T,E,ds,dt,D,r,option):
    N = int(S_star/ds)
    M = int(T/dt) + 1
    
    V = np.zeros((N+1,M+1)) # build a zero matrix
    #Final Payoff Condition (t == T)
    for n in range(N+1):
        V[n][M] = payoff(n*ds,E,option)
    
    a_n = np.zeros(N)
    b_n = np.zeros(N)
    c_n = np.zeros(N)
    
    #setup abc elements
    for n in range(1,N,1):
        a_n[n] = -0.5*((sigma**2)*((n)**2)-(n)*(r-D))*dt
    a_n = np.delete(a_n,0)
    for n in range(0,N,1): 
        b_n[n] = 1+((sigma**2)*(n**2)+r)*dt
        c_n[n] = -0.5*((sigma**2)*(n**2)+n*(r-D))*dt
    
    #create an abc metrix
    abc = np.diag(np.append(b_n,0)) + np.diag(c_n,1) + np.diag(np.append(a_n,0),-1)
    abc[N][N] = 1-(N*(r-D)-r)*dt #b_N
    abc[N][N-1] = N*(r-D)*dt #a_N

    #solve the matrix system
    for m in range(M,0,-1):
        V[:,m-1] = np.linalg.solve(abc,V[:,m])
    
    return V

## The implicit scheme for American options
For American options, they are the same as the explicit scheme about the no-arbitrage argument, so we have the same condition. 

$$ V \ge  payoff(S,E) $$



In [10]:
# American Options, Implicit Scheme
def FDM_BS_American_Im(T,E,ds,dt,D,r,option):
    N = int(S_star/ds)
    M = int(T/dt) + 1
    
    V = np.zeros((N+1,M+1)) # build a zero matrix
    #Final Payoff Condition (t == T)
    for n in range(N+1):
        V[n][M] = payoff(n*ds,E,option)
    
    a_n = np.zeros(N)
    b_n = np.zeros(N)
    c_n = np.zeros(N)
    
    #setup abc elements
    for n in range(1,N,1):
        a_n[n] = -0.5*((sigma**2)*((n)**2)-(n)*(r-D))*dt
    a_n = np.delete(a_n,0)
    for n in range(0,N,1): 
        b_n[n] = 1+((sigma**2)*(n**2)+r)*dt
        c_n[n] = -0.5*((sigma**2)*(n**2)+n*(r-D))*dt
    
    #create an abc metrix
    abc = np.diag(np.append(b_n,0)) + np.diag(c_n,1) + np.diag(np.append(a_n,0),-1)
    abc[N][N] = 1-(N*(r-D)-r)*dt #b_N
    abc[N][N-1] = N*(r-D)*dt #a_N

    #solve the matrix system
    for m in range(M,0,-1):
        V[:,m-1] = np.linalg.solve(abc,V[:,m])

        for n in range(0,N+1,1):
            b_0 = 1-(r*dt)
            V[n,m] = np.maximum( payoff(n*ds,E,option)*b_0 , V[n,m] )
        
    return V

## The $\theta$-Method
It is the combination of the two previous scheme.

<!-- %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% add pic %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% -->

$$ \frac{V^{m+1}_n-V^m_n}{\delta t} = \theta\frac{V^{m+1}_{n+1}-2V^m_n+V^{m+1}_{n-1}}{\delta S^2}+(1-\theta)\frac{V^m_{n+1}-2V^m_n+V^m_{n-1}}{\delta S^2}$$

$$ a_n V^{m-1}_{n-1} + b_n V^{m-1}_n + c_n V^{m-1}_{n+1} = A_n V^{m}_{n-1} + B_n V^{m}_n + C_n V^{m}_{n+1} $$

$$
\begin{aligned}
    a_n &= -\theta\frac{1}{2}(sigma^2n^2-n(r-D))\delta t\\
    b_n &= 1+\theta(sigma^2n^2+r)\delta t\\
    c_n &= -\theta\frac{1}{2}(sigma^2n^2+n(r-D))\delta t\\
    A_n &= (1-\theta)\frac{1}{2}(sigma^2n^2-n(r-D))\delta t\\
    B_n &= 1-(1-\theta)(sigma^2n^2+r)\delta t\\
    C_n &= (1-\theta)\frac{1}{2}(sigma^2n^2+n(r-D))\delta t\\
\end{aligned}
$$


In [20]:
# European Options, The Theta-Method
def FDM_BS_European_TM(T,E,ds,dt,D,r,option):
    theta = 0.5
    
    N = int(S_star/ds)
    M = int(T/dt) + 1
    
    V = np.zeros((N+1,M+1)) # build a zero matrix
    U = np.zeros((N+1,M+1)) # build a zero matrix
    
    #Final Payoff Condition (t == T)
    for n in range(N+1):
        V[n][M] = payoff(n*ds,E,option)
        U[n][M] = payoff(n*ds,E,option)
    
    a_n = np.empty(N)
    b_n = np.empty(N)
    c_n = np.empty(N)

    A_n = np.empty(N)
    B_n = np.empty(N)
    C_n = np.empty(N)

    #setup ABC elements
    for n in range(1,N,1):
        A_n[n] = 0.5*(1-theta)*((sigma**2)*((n)**2)-(n)*(r-D))*dt
    A_n = np.delete(A_n,0)

    for n in range(0,N,1): 
        B_n[n] = 1-(1-theta)*((sigma**2)*(n**2)+r)*dt
        C_n[n] = 0.5*(1-theta)*((sigma**2)*(n**2)+n*(r-D))*dt

    #create an ABC matrix
    U = np.diag(np.append(B_n,0)) + np.diag(C_n,1) + np.diag(np.append(A_n,0),-1)
    U[N][N] = 1+(1-theta)*(N*(r-D)-r)*dt
    U[N][N-1] = -(1-theta)*N*(r-D)*dt


    #setup abc elements
    for n in range(1,N,1):
        a_n[n] = -0.5*theta*((sigma**2)*((n)**2)-(n)*(r-D))*dt
    a_n = np.delete(a_n,0)

    for n in range(0,N,1): 
        b_n[n] = 1+theta*((sigma**2)*(n**2)+r)*dt
        c_n[n] = -0.5*theta*((sigma**2)*(n**2)+n*(r-D))*dt

    #create an abc matrix
    abc = np.diag(np.append(b_n,0)) + np.diag(c_n,1) + np.diag(np.append(a_n,0),-1)
    abc[N][N] = 1-theta*(N*(r-D)-r)*dt
    abc[N][N-1] = theta*N*(r-D)*dt

    for m in range(M,0,-1):
        V[:,m-1] = np.linalg.solve(np.linalg.inv(U) @ abc,V[:,m])

    return V

In [24]:
# American Options, The Theta-Method
def FDM_BS_American_TM(T,E,ds,dt,D,r,option):
    theta = 0.5
    
    N = int(S_star/ds)
    M = int(T/dt) + 1
    
    V = np.zeros((N+1,M+1)) # build a zero matrix
    U = np.zeros((N+1,M+1)) # build a zero matrix
    
    #Final Payoff Condition (t == T)
    for n in range(N+1):
        V[n][M] = payoff(n*ds,E,option)
        U[n][M] = payoff(n*ds,E,option)
    
    a_n = np.empty(N)
    b_n = np.empty(N)
    c_n = np.empty(N)

    A_n = np.empty(N)
    B_n = np.empty(N)
    C_n = np.empty(N)

    #setup ABC elements
    for n in range(1,N,1):
        A_n[n] = 0.5*(1-theta)*((sigma**2)*((n)**2)-(n)*(r-D))*dt
    A_n = np.delete(A_n,0)

    for n in range(0,N,1): 
        B_n[n] = 1-(1-theta)*((sigma**2)*(n**2)+r)*dt
        C_n[n] = 0.5*(1-theta)*((sigma**2)*(n**2)+n*(r-D))*dt

    #create an ABC matrix
    U = np.diag(np.append(B_n,0)) + np.diag(C_n,1) + np.diag(np.append(A_n,0),-1)
    U[N][N] = 1+(1-theta)*(N*(r-D)-r)*dt
    U[N][N-1] = -(1-theta)*N*(r-D)*dt


    #setup abc elements
    for n in range(1,N,1):
        a_n[n] = -0.5*theta*((sigma**2)*((n)**2)-(n)*(r-D))*dt
    a_n = np.delete(a_n,0)

    for n in range(0,N,1): 
        b_n[n] = 1+theta*((sigma**2)*(n**2)+r)*dt
        c_n[n] = -0.5*theta*((sigma**2)*(n**2)+n*(r-D))*dt

    #create an abc matrix
    abc = np.diag(np.append(b_n,0)) + np.diag(c_n,1) + np.diag(np.append(a_n,0),-1)
    abc[N][N] = 1-theta*(N*(r-D)-r)*dt
    abc[N][N-1] = theta*N*(r-D)*dt

    for m in range(M,0,-1):
        V[:,m-1] = np.linalg.solve(np.linalg.inv(U) @ abc,V[:,m])

        for n in range(0,N+1,1):
            b_0 = 1-(r*dt)
            V[n,m] = np.maximum( payoff(n*ds,E,option)*b_0 , V[n,m])
            
    return V

In [27]:
result_ET = FDM_BS_European_TM(T,E,ds,dt,D,r,"call")
# print('\n'.join([' '.join(['{0:0.3f}'.format(item) for item in row]) for row in result_ET-result_E]))

result_AT = FDM_BS_American_TM(T,E,ds,dt,D,r,"call")
# print('\n'.join([' '.join(['{0:0.3f}'.format(item) for item in row]) for row in result_AT-result_A]))

https://www.sciencedirect.com/science/article/abs/pii/B9780080965277000088